In [ ]:
import sys
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
import pandas as pd
import numpy as np
import glob, os

import itertools

from backtester.exoinfo import EXOInfo
from exobuilder.data.exostorage import EXOStorage
from IPython.core.display import display, HTML

import statsmodels.tsa.api as smt
import statsmodels.tsa.filters as smfilters

import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go

import pandas as pd
import numpy as np
import scipy
import pprint

%pylab inline
figsize(20,10)

In [ ]:
# Loading global setting for MongoDB etc.
from scripts.settings import *

try:
    from scripts.settings_local import *
except:
    pass

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)
pp = pprint.PrettyPrinter(indent=4)

exo_list = storage.exo_list()
pp.pprint(exo_list)


df = pd.DataFrame()

for e in exo_list:
    exo_series, exo_dict = storage.load_series(e)
    
    exo_series = exo_series[exo_series.columns[0]]
    df[exo_dict['name']] = exo_series
#exo_series.plot()

In [ ]:
df.tail()

In [ ]:
px_ser = df.ES_CallSpread.dropna()
#px_ser = (px_ser - px_ser.ewm(2).mean()).dropna()

sd = smt.seasonal_decompose(px_ser.asfreq('B').ffill().dropna().iloc[:])#.plot()

resid = sd.resid

In [ ]:
#px_ser.plot()

px_ser.ewm(10).mean().ewm(10).mean().ewm(10).mean().ewm(10).mean().ewm(10).mean().plot()
px_ser.ewm(50).mean().plot()
fixed_hpfilt.plot()
#px_ser.ewm(10).mean().ewm(10).mean().plot()
#px_ser.ewm(10).mean().ewm(10).mean().ewm(10).mean().plot()
#px_ser.ewm(10).mean().ewm(10).mean().ewm(10).mean().ewm(10).mean().plot()
#px_ser.rolling(50, center=True, win_type='blackmanharris').mean().plot()

In [ ]:
centered_ma = px_ser.rolling(100, center=True, win_type='blackmanharris').mean()
centered_ma[-1] = px_ser[-1]

centered_ma_trend = centered_ma.interpolate('cubic')

centered_ma_trend.ix['2016'].plot()

px_ser.rolling(100, center=True, win_type='blackmanharris').mean().ix['2016'].plot()
px_ser.ix['2016'].plot()

In [ ]:
maema_trend = (px_ser.rolling(50, center=True, win_type='blackmanharris').mean().ewm(10).mean())

px_ser.plot()
maema_trend.plot()
#(px_ser - px_ser.ewm(30).mean()).plot()
#(px_ser - px_ser.shift(10)).plot()
#resid.plot()

In [ ]:
#smfilters.hp_filter.hpfilter(px_ser)[1]#.plot()
hpfilt = smfilters.api.hpfilter(px_ser, lamb=50000)[1]#.plot()

#smt.filters.
hpfilt.plot()

#resid.plot()
(px_ser - hpfilt).plot()
px_ser.plot()

In [ ]:
hpfilt.ix['2015'].plot()

px_ser.ix['2015'].plot()

In [ ]:
maema_trend.plot()

centered_ma_trend.plot()
hpfilt.plot()

In [ ]:
#smfilters.hp_filter.hpfilter(px_ser)[1]#.plot()
cycle, trend = smfilters.bk_filter.bkfilter(px_ser, low=2, high=60)#.plot()

trend.plot()

#resid.plot()
#(px_ser - filt).plot()
px_ser.plot()

In [ ]:
smfilters.bk_filter.bkfilter(px_ser, low=2, high=60)

In [ ]:
smfilters.bk_filter.bkfilter(px_ser, low=2, high=60).plot()

In [ ]:
#smfilters.hp_filter.hpfilter(px_ser)[1]#.plot()
cycle, trend = smfilters.cf_filter.cffilter(px_ser, low=2, high=60, drift=False)#.plot()

trend.plot()
#trend.expanding().mean().plot()

cycle.plot()
resid.plot()
#(px_ser - filt).plot()
px_ser.plot()

In [ ]:
#resid.plot()
(px_ser - px_ser.shift(1)).plot()
px_ser.plot()
#axhline(0)

In [ ]:
# Detrend func makes a px series pretty stationary
detrended_px = pd.Series(smt.tsatools.detrend(px_ser, order=1))
detrended_px.plot()
detrended_px.expanding().mean().plot()
detrended_px.expanding().std().plot()

detrended_px.expanding().max().plot()
detrended_px.expanding().min().plot()

In [ ]:
resid.plot()
resid.expanding().mean().plot()
resid.expanding().std().plot()

resid.expanding().max().plot()
resid.expanding().min().plot()

In [ ]:
sd = smt.seasonal_decompose(px_ser.asfreq('W').ffill().dropna().iloc[:])#.plot()
sd.plot()

In [ ]:
pd.Series((sd.trend + sd.resid), name='Trend+Resid.').plot(legend=True)
pd.Series((sd.observed), name='Observed').plot(legend=True)

# Entry Rules
# Exit rules - resid got back to 0, seasonal cycle got back to min/max
# Also Monthly or Weekly TF may be useful
# Resid is a detrended price series

In [ ]:
resid = sd.resid
seas = sd.seasonal

resid.plot()
sd.observed.plot(secondary_y=True)

In [ ]:
resid.cumsum().plot()
resid.plot()

In [ ]:
# Observed - Trend
(sd.observed - sd.trend).plot()

In [ ]:
## Persent rank >= value or <= value
resid_pctrank = resid.rank(pct=True)

resid.plot()
resid_pctrank.plot(secondary_y=True)

axhline(0.5)
axhline(0.9)
axhline(0.1)

In [ ]:
## Z Score >= value or <= value
# value, and period are subjects of optimization

resid_zscore = (resid - resid.rolling(20).mean()) / resid.rolling(20).std()

resid.plot()
resid_zscore.plot(secondary_y=True)

axhline(0)
axhline(3)
axhline(-2)

In [ ]:
seas.plot()

In [ ]:
sd.observed.plot()
(seas == seas.max()).plot(secondary_y=True)

In [ ]:
#df.join(pd.Series(seas == seas.max(), name='sig'))

In [ ]:
sd.observed.plot()
(seas == seas.min()).plot(secondary_y=True)
#((seas.shift(1) < 0) & ((seas >= 0))).plot(secondary_y=True)

In [ ]:
sd.observed.plot()
(seas == seas.max()).plot(secondary_y=True)
(seas == seas.min()).plot(secondary_y=True)


# Streaming























In [ ]:
#py.sign_in('tmqr', '8rctfyu197')
py.sign_in('dmveden', 'rzbs7xw8ft')

tls.set_credentials_file(username='dmveden', api_key='rzbs7xw8ft')

In [ ]:
stream_ids = tls.get_credentials_file()['stream_ids']
print(stream_ids)

In [ ]:
# Get stream id from stream id list 
stream_id1 = 'twfcm3v44s'
stream_id2 = '5dnfr3e5d8'
stream_id3 = '1ynb4ujpbt'

# Make instance of stream id object 
stream_1 = go.Stream(
    token=stream_id1,  # link stream id to 'token' key
    maxpoints=200      # keep a max of 80 pts on screen
)

# Make instance of stream id object 
stream_2 = go.Stream(
    token=stream_id2,  # link stream id to 'token' key
    maxpoints=200      # keep a max of 80 pts on screen
)

# Make instance of stream id object 
stream_3 = go.Stream(
    token=stream_id3,  # link stream id to 'token' key
    maxpoints=200      # keep a max of 80 pts on screen
)

# Initialize trace of streaming plot by embedding the unique stream_id
trace1 = go.Scatter(
    x=[],
    y=[],
    mode='lines+markers',
    stream=stream_1         # (!) embed stream id, 1 per trace
)

trace1 = go.Scatter(x=[], y=[], stream=stream_1, name='Fixed_filter')
trace2 = go.Scatter(x=[], y=[], stream=stream_2, name='Price', marker=dict(color='rgb(148, 103, 189)'))
trace3 = go.Scatter(x=[], y=[], stream=stream_3, name='Standard filter', marker=dict(color='rgb(255, 0, 0'))

data = go.Data([trace1, trace2, trace3])

# Add title to layout object
layout = go.Layout(title='Hodrick Prescott Filter Calculation Process')

# Make a figure object
fig = go.Figure(data=data, layout=layout)

# Send fig to Plotly, initialize streaming plot, open new tab
py.iplot(fig, filename='python-streaming')

In [ ]:
# We will provide the stream link object the same token that's associated with the trace we wish to stream to
s_1 = py.Stream(stream_id=stream_id1)
s_2 = py.Stream(stream_id=stream_id2)
s_3 = py.Stream(stream_id=stream_id3)

# We then open a connection
s_1.open()
s_2.open()
s_3.open()

In [ ]:
# (*) Import module keep track and format current time
import datetime 
import time   
 
i = 10   # a counter
k = 5    # some shape parameter

# Delay start of stream by 5 sec (time to switch tabs)
#time.sleep(2) 

#hpfilt = smfilters.api.hpfilter(px_ser, lamb=50000)[1]#.plot()
#resid = (px_ser - hpfilt)
fixed_hpfilt = pd.Series(NaN, index=px_ser.index)
first_time_calc_flag = True

while True:
    if i > px_ser.size:
        break
        
    hpfilt = smfilters.api.hpfilter(px_ser.iloc[:i], lamb=50000)[1]#.plot()
    #filt = smfilters.cf_filter.cffilter(px_ser.iloc[:i], low=2, high=60, drift=False)[1]
    #hpfilt = px_ser.iloc[:i].rolling(30, center=True).mean()
    #filt[-1] = px_ser.iloc[:i][-1]
    #filt.interpolate(inplace=True)
    
    
     
    hpfilt_slice = smfilters.api.hpfilter(px_ser.iloc[:i], lamb=50000)[1]
    fixed_hpfilt.iloc[i] = np.mean(hpfilt_slice[-1:])
    
    filt = fixed_hpfilt
    
    # Current time on x-axis, random numbers on y-axis
    x_1 = filt.iloc[:i].index
    y_1 = filt.iloc[:i]
     
    x_2 = px_ser.iloc[:i].index
    y_2 = px_ser.iloc[:i] 
    
    x_3 = hpfilt.iloc[:i].index
    y_3 = hpfilt.iloc[:i] 
    
    # Send data to your plot
    s_1.write(dict(x=x_1, y=y_1))  
    s_1.write(dict(x=x_1, y=y_1)) 
    
    s_2.write(dict(x=x_2, y=y_2))  
    s_2.write(dict(x=x_2, y=y_2)) 
    
    s_3.write(dict(x=x_3, y=y_3))  
    s_3.write(dict(x=x_3, y=y_3)) 
    #     Write numbers to stream to append current data on plot,
    #     write lists to overwrite existing data on plot
            
    time.sleep(0.1)  # plot a point every second    
    i += 1
    
   
# Close the stream when done plotting
s_1.close() 
s_2.close() 

In [ ]:
hpfilt = smfilters.api.hpfilter(px_ser, lamb=50000)[0]

def x():
    fixed_hpfilt = pd.Series(NaN, index=px_ser.index)

    window = 10
    first_time_calc_flag = True

    for i in range(window, len(px_ser)):
        hpfilt_slice = smfilters.api.hpfilter(px_ser.iloc[:i], lamb=50000)[0]
        fixed_hpfilt.iloc[i] = np.mean(hpfilt_slice[-1:])
        
    return fixed_hpfilt

In [ ]:
%timeit x()

In [ ]:
px_ser.ix[:].plot()
#px_ser.rolling(100).std().ix["2011":].plot()
#hpfilt.ix['2011':].plot()
fixed_hpfilt.plot()
fixedfilter_resid.plot()

In [ ]:
fixedfilter_resid = px_ser - fixed_hpfilt

fixedfilter_resid.plot()
fixedfilter_resid.expanding().mean().plot()
fixedfilter_resid.expanding().std().plot()

fixedfilter_resid.expanding().max().plot()
fixedfilter_resid.expanding().min().plot()

In [ ]:
hpfilt_resid = px_ser - hpfilt

hpfilt_resid.plot()
hpfilt_resid.expanding().mean().plot()
hpfilt_resid.expanding().std().plot()

hpfilt_resid.expanding().max().plot()
hpfilt_resid.expanding().min().plot()

In [ ]:
hpfilt_resid.plot()
fixedfilter_resid.plot()

In [ ]:
(fixedfilter_resid - hpfilt_resid).plot()